# Recurrent Neural Network Setup

**Recurrent Neural Networks (RNNs)** are very effective for Natural Language Processing and other sequence tasks because they have "memory". They can read inputs $x^{\langle t \rangle}$ one at a time, and remember information/context through the hidden layer activations that get passed from one time-step to the next. This allows a unidirectional RNN to take information from the past to process later inputs. A bidirectional RNN can take context from both the past and the future. 

In [1]:
import numpy as np
from rnn_utils import *

## 1. Forward Propagation

### Input $x$

* $n_x$ = number of units in a single training example
* $m$ = number of training examples per mini-batch
* $T_x$ = number of time steps in input


1. A single input example, $x^{(i)}$, is a one-dimensional input vector, such as a one-hot encoded vector with $n_x$ units. So $x^{(i)}$ would have the shape of ($n_x$,).  
2. If we have mini-batches of 20 training examples, we can stack 20 columns of $x^{(i)}$ examples into a matrix. Our tensor is now $(n_x,m)$.
3. A recurrent neural network has multiple time steps, indexed with $t$, and so $x$ becomes $(n_x,m,T_x)$.
4. At each time step, we use a mini-batch of training examples. So, for each time step $t$, we use a 2D slice of shape $x^{\langle t \rangle}$ = $(n_x,m)$.


### Hidden State $a$

* $n_x$ = number of units in a single hidden activation layer

The **hidden state** is the activation $a^{\langle t \rangle}$ that is passed to the RNN from one time step to another. Similar to $x$, the shape of the hidden state is $(n_{a}, m, T_x)$. Each 2D slice $a^{\langle t \rangle}$ is of shape $(n_{a}, m)$. 

### Prediction $\hat{y}$

* $n_{y}$: number of units in the vector representing the prediction
* $m$: number of examples in a mini-batch
* $T_{y}$: number of time steps in the prediction

$\hat{y}$ is also of shape $(n_{y}, m, T_{y})$, with 2D slice $\hat{y}^{\langle t \rangle}$ having shape $(n_{y}, m)$.


### RNN Implementation Process
1. Implement the calculations needed for one time-step of the RNN.
2. Implement a loop over $T_x$ time-steps in order to process all the inputs, one at a time. 

## 1.1 - RNN Cell

<img src="images/rnn_step_forward_figure2_v3a.png" style="width:700px;height:300px;">

**`rnn_cell_forward() Overview`**:

`Implements a single forward step of an RNN cell.`

**`Arguments`**
```
xt = input data at t                                 (n_x, m)
a_prev = hidden state at t-1                         (n_a, m)
parameters = python dictionary containing:
                Wax = input weight                   (n_a, n_x)
                Waa = hidden state weight            (n_a, n_a)
                Wya = hidden-state > output weight   (n_y, n_a)
                ba =  inputs bias                    (n_a, 1)
                by = hidden-state > output bias      (n_y, 1)
```
**`Returns`**
```
a_next = next hidden state                           (n_a, m)
yt_pred = prediction at t,                           (n_y, m)
cache = tuple of values needed for backprop          (a_next, a_prev, xt, parameters)
```

**`Formulas`**

<center>$a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$</center>

<center>$\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$</center>

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)    # compute next activation state
    yt_pred = softmax(np.dot(Wya, a_next) + by)                     # compute output of current cell
    cache = (a_next, a_prev, xt, parameters)                        # store values for backprop
    
    return a_next, yt_pred, cache

## 1.2 - RNN forward pass 

RNNs are essentially RNN cell repetitions, so we re-use the RNN cell T_x times. Weights and biases are re-used each time step. 

**`rnn_forward() Overview`**:

`Implement the forward propagation of a recurrent neural network.`

**`Arguments`**
```
x = input                                                              (n_x, m, T_x).
a0 = initial hidden state                                              (n_a, m)
parameters = dictionary of Waa, Wax, Wya, ba, by
```
**`Returns`**
```
a = hidden states                                                      (n_a, m, T_x)
y_pred = predictions                                                   (n_y, m, T_x)
caches = tuple of values for backprop                                  (list of caches, x)
```

In [3]:
def rnn_forward(x, a0, parameters):
    
    # initialize caches and retrieve dimensions
    caches = []
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
        
    # initialize a, y_pred, and a_next
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    a_next = a0
    
    for t in range(T_x):
        # run single RNN cell at t
        xt = x[:,:,t]
        a_next, yt_pred, cache = rnn_cell_forward(xt, a_next, parameters)
        a[:,:,t] = a_next          # save new next hidden state in a
        y_pred[:,:,t] = yt_pred    # save prediction in y
        caches.append(cache)       # append cache to caches

    caches = (caches, x)           # cache for backprop
    
    return a, y_pred, caches

## 2. Long Short-Term Memory (LSTM)

<img src="images/LSTM_figure4_v3a.png" style="width:500;height:400px;">
<caption><center> **Figure 4**: LSTM-cell. This tracks and updates a "cell state" or memory variable $c^{\langle t \rangle}$ at every time-step, which can be different from $a^{\langle t \rangle}$. </center></caption>

### Overview of gates and states

The **forget gate** $\mathbf{\Gamma}_{f}$ is a tensor that contains values between 0 and 1, used to "forget" an outdated state.
* If a unit in the forget gate is close to 0, the LSTM will "forget" the stored state in the corresponding unit of the previous cell state.
* If a unit in the forget gate is close to 1, the LSTM will mostly remember the corresponding value in the stored state.

$$\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)\tag{1} $$

The **candidate value** $\tilde{\mathbf{c}}^{\langle t \rangle}$ is a tensor containing values from -1 to 1, with information from the current time step that *may* be stored in the current cell state $\mathbf{c}^{\langle t \rangle}$. Which parts of the candidate value get passed on depends on the update gate.

$$\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{a}^{\langle t - 1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right) \tag{3}$$

The **update gate** $\mathbf{\Gamma}_{i}$ decides what aspects of $\tilde{\mathbf{c}}^{\langle t \rangle}$ to add to $c^{\langle t \rangle}$. It is a tensor containing values between 0 and 1.
* When a unit in the update gate is close to 1, it allows the value of the candidate $\tilde{\mathbf{c}}^{\langle t \rangle}$ to be passed onto the hidden state $\mathbf{c}^{\langle t \rangle}$
* When a unit in the update gate is close to 0, it prevents the corresponding value in the candidate from being passed onto the hidden state.

$$\mathbf{\Gamma}_i^{\langle t \rangle} = \sigma(\mathbf{W}_i[a^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)\tag{2} $$ 

The **cell state** $\mathbf{c}^{\langle t \rangle}$ is the memory that gets passed onto future time steps, and a combination of the previous cell state and the candidate value.

$$ \mathbf{c}^{\langle t \rangle} = \mathbf{\Gamma}_f^{\langle t \rangle}* \mathbf{c}^{\langle t-1 \rangle} + \mathbf{\Gamma}_{i}^{\langle t \rangle} *\mathbf{\tilde{c}}^{\langle t \rangle} \tag{4} $$

The **output gate** $\mathbf{\Gamma}_{o}$ decides what gets sent as the prediction (output) of the time step, containing values that range from 0 to 1.

$$ \mathbf{\Gamma}_o^{\langle t \rangle}=  \sigma(\mathbf{W}_o[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})\tag{5}$$ 

The **hidden state** $\mathbf{a}^{\langle t \rangle}$ gets passed to the LSTM cell's next time step. It is used to determine the three gates ($\mathbf{\Gamma}_{f}, \mathbf{\Gamma}_{u}, \mathbf{\Gamma}_{o}$) of the next time step, and $y^{\langle t \rangle}$.

$$ \mathbf{a}^{\langle t \rangle} = \mathbf{\Gamma}_o^{\langle t \rangle} * \tanh(\mathbf{c}^{\langle t \rangle})\tag{6} $$

Since the **prediction** $\mathbf{y}^{\langle t \rangle}_{pred}$ in this case is a classification, we use softmax.

$$\mathbf{y}^{\langle t \rangle}_{pred} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{a}^{\langle t \rangle} + \mathbf{b}_{y})$$

### 2.1 - LSTM cell

**`lstm_cell_forward() Overview`**:

`Implement a single forward step of the LSTM-cell.`

**`Arguments`**
```
xt = input data at t            (n_x, m)
a_prev = hidden state at t-1    (n_a, m)
c_prev = memory state at t-1    (n_a, m)
parameters = dictionary containing:
                Wf = forget gate weight          (n_a, n_a + n_x)
                bf = forget gate bias            (n_a, 1)
                Wi = update gate weight          (n_a, n_a + n_x)
                bi = update gate bias            (n_a, 1)
                Wc = candidate weight            (n_a, n_a + n_x)
                bc = candidate bias              (n_a, 1)
                Wo = output gate weight          (n_a, n_a + n_x)
                bo = output gate bias            (n_a, 1)
                Wy = output weight               (n_y, n_a)
                by = output bias                 (n_y, 1)
```
**`Returns`**
```
a_next = next hidden state       (n_a, m)
c_next = next memory state       (n_a, m)
yt_pred = prediction at t        (n_y, m)
cache = tuple for backprop       (a_next, c_next, a_prev, c_prev, xt, parameters)

ft/it/ot = forget/update/output gates
cct = candidate value
c = cell state
```

In [7]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):

    Wf = parameters["Wf"] # forget gate
    bf = parameters["bf"]
    Wi = parameters["Wi"] # update gate
    bi = parameters["bi"]
    Wc = parameters["Wc"] # candidate value
    bc = parameters["bc"]
    Wo = parameters["Wo"] # output gate
    bo = parameters["bo"]
    Wy = parameters["Wy"] # prediction
    by = parameters["by"]
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    concat = np.concatenate((a_prev, xt), axis=0)
    ft = sigmoid(np.dot(Wf, concat) + bf)        # forget gate
    it = sigmoid(np.dot(Wi, concat) + bi)        # update gate
    cct = np.tanh(np.dot(Wc, concat) + bc)       # candidate value
    c_next = it * cct + ft * c_prev              # cell state
    ot = sigmoid(np.dot(Wo, concat) + bo)        # output gate
    a_next = ot * np.tanh(c_next)                # hidden state
    
    yt_pred = softmax(np.dot(Wy, a_next) + by)   # compute LSTM cell prediction
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

### 2.2 - Forward pass for LSTM

**`lstm_forward() Overview`**

`Implement the forward propagation of the recurrent neural network using an LSTM-cell.`

**`Arguments`**
```
x = input (n_x, m, T_x).
a0 = initial hidden state (n_a, m)
parameters = dictionary of parameters
```
**`Returns`**
```
a = hidden states (n_a, m, T_x)
y = predictions (n_y, m, T_x)
c = cell states (n_a, m, T_x)
caches = (list of all the caches, x)
```

In [13]:
def lstm_forward(x, a0, parameters):

    caches = []
    Wy = parameters['Wy']
    n_x, m, T_x = x.shape
    n_y, n_a = Wy.shape
    
    # initialize a, c, y and a_next, c_next
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    a_next = a0
    c_next = np.zeros(a_next.shape)
    
    for t in range(T_x):
        xt = x[:,:,t]
        a_next, c_next, yt, cache = lstm_cell_forward(xt, a_next, c_next, parameters)
        a[:,:,t] = a_next
        c[:,:,t]  = c_next
        y[:,:,t] = yt
        caches.append(cache)
            
    caches = (caches, x)

    return a, y, c, caches